In [1]:
import pandas as pd
import numpy as np
import itertools
import tqdm
%load_ext Cython

In [2]:
print("Read people.csv...")
people=pd.read_csv('people.csv',
                   dtype={'char_38': np.int32},
                   parse_dates=['date'])

print("Load act_train.csv...")
act_train=pd.read_csv('act_train.csv',
                      parse_dates=['date'])

print("Load act_test.csv...")
act_test=pd.read_csv('act_test.csv',
                     parse_dates=['date'])


print("Process tables...")
for table in [act_train, act_test]:
    table['year'] = table['date'].dt.year
    table['month'] = table['date'].dt.month
    table['day'] = table['date'].dt.day
    table['weekday'] = table['date'].dt.weekday
    ##table.drop('date', axis=1, inplace=True)
    table['activity_category'] = table['activity_category'].str.lstrip('type ').astype(np.int32)
    for i in range(1, 11):
        table['char_' + str(i)].fillna('type -999', inplace=True)
        table['char_' + str(i)] = table['char_' + str(i)].str.lstrip('type ').astype(np.int32)
    table['people_id'] = table['people_id'].str.lstrip('ppl_').astype(np.float).astype(np.int32)

people['year'] = people['date'].dt.year
people['month'] = people['date'].dt.month
people['day'] = people['date'].dt.day
people['weekday'] = people['date'].dt.weekday

#people.drop('date', axis=1, inplace=True)
people['group_1'] = people['group_1'].str.lstrip('group ').astype(np.int32)
people['people_id'] = people['people_id'].str.lstrip('ppl_').astype(np.float).astype(np.int32)
for i in range(1, 10):
    people['char_' + str(i)] = people['char_' + str(i)].str.lstrip('type ').astype(np.int32)
for i in range(10, 38):
    people['char_' + str(i)] = people['char_' + str(i)].astype(np.int32)

Read people.csv...
Load act_train.csv...
Load act_test.csv...
Process tables...


In [4]:
for name in people.columns.values:
    print('%s: %d' % (name, len(people[name].unique())))
print('')    
for name in act_train.columns.values:
    print('%s: %d' % (name, len(act_train[name].unique())))
print('')    
for name in act_test.columns.values:
    print('%s: %d' % (name, len(act_test[name].unique())))

people_id: 189118
char_1: 2
group_1: 34224
char_2: 3
date: 1196
char_3: 43
char_4: 25
char_5: 9
char_6: 7
char_7: 25
char_8: 8
char_9: 9
char_10: 2
char_11: 2
char_12: 2
char_13: 2
char_14: 2
char_15: 2
char_16: 2
char_17: 2
char_18: 2
char_19: 2
char_20: 2
char_21: 2
char_22: 2
char_23: 2
char_24: 2
char_25: 2
char_26: 2
char_27: 2
char_28: 2
char_29: 2
char_30: 2
char_31: 2
char_32: 2
char_33: 2
char_34: 2
char_35: 2
char_36: 2
char_37: 2
char_38: 101
year: 4
month: 12
day: 31
weekday: 7

people_id: 151295
activity_id: 2197291
date: 411
activity_category: 7
char_1: 52
char_2: 33
char_3: 12
char_4: 8
char_5: 8
char_6: 6
char_7: 9
char_8: 19
char_9: 20
char_10: 6516
outcome: 2
year: 2
month: 12
day: 31
weekday: 7

people_id: 37823
activity_id: 498687
date: 411
activity_category: 7
char_1: 49
char_2: 32
char_3: 12
char_4: 8
char_5: 7
char_6: 6
char_7: 9
char_8: 19
char_9: 20
char_10: 3962
year: 2
month: 12
day: 31
weekday: 7


In [3]:
train=pd.merge(people, act_train, on='people_id')
test=pd.merge(people, act_test, on='people_id')
del(act_train)
del(act_test)

In [4]:
def mutual_entropy(a, l):
    n = len(a)
    a_ent = -np.sum([x*np.log(x) for x in a.value_counts()/n])
    l_ent = -np.sum([x*np.log(x) for x in l.value_counts()/n])
    al_ent = -np.sum([x*np.log(x) for x in a[l==0].value_counts()/n]) - \
             np.sum([x*np.log(x) for x in a[l==1].value_counts()/n])
    return a_ent + l_ent - al_ent

def iteraction_entropy(a, b, l):
    n = len(a)
    aa=np.array(a)
    bb=np.array(b)
    ll=np.array(l)
    ab = pd.Series([(aa[i], bb[i]) for i in np.arange(0, n)])
    abl_ent=-np.sum([x*np.log(x) for x in ab[ll==0].value_counts()/n]) - \
            np.sum([x*np.log(x) for x in ab[ll==1].value_counts()/n])
    ab_ent =-np.sum([x*np.log(x) for x in ab.value_counts()/ n])
    l_ent = -np.sum([x*np.log(x) for x in l.value_counts()/n])
    a_ent = -np.sum([x*np.log(x) for x in a.value_counts()/n])
    b_ent = -np.sum([x*np.log(x) for x in b.value_counts()/n])
    abl_ment = ab_ent + l_ent - abl_ent
    return abl_ment - mutual_entropy(a, l) - mutual_entropy(b, l) , ab_ent - a_ent - b_ent

In [5]:
%%cython
import numpy as np
import pandas as pd
def get_log_posterior(train, name):
    s0=train[train['outcome'] == 0][name]
    s0=s0.append(pd.DataFrame({0:train[name].unique()}))[0]
    s0=-np.log(s0.value_counts(dropna=False)/len(s0))
    s0=dict(s0)
    s1=train[train['outcome'] == 1][name]    
    s1=s1.append(pd.DataFrame({0:train[name].unique()}))[0]
    s1=-np.log(s1.value_counts(dropna=False)/len(s1))
    s1=dict(s1)
    return s0,s1


In [8]:
a=dict()
b=dict()
gain=dict()
for name in train.columns.values:
    if name not in set(['people_id', 'activity_id', 'char_38']):
        print(name, end=': ')
        gain[name] = mutual_entropy(train[name], train['outcome'])
        print(gain[name])
        a[name], b[name] = get_log_posterior(train, name)

char_1_x: 0.0128333413646
group_1: 0.623434548023
char_2_x: 0.29692885141
date_x: 0.0947567469661
char_3_x: 0.0248090040523
char_4_x: 0.0137991158875
char_5_x: 0.0137259575534
char_6_x: 0.0449202668956
char_7_x: 0.075758721909
char_8_x: 0.091276730308
char_9_x: 0.0818365865575
char_10_x: 0.0363928982184
char_11: 0.0256503268109
char_12: 0.0253169791715
char_13: 0.052352334771
char_14: 0.0307289452823
char_15: 0.0349814003224
char_16: 0.03903751061
char_17: 0.0414109644556
char_18: 0.022403188299
char_19: 0.0392695008341
char_20: 0.0369277498289
char_21: 0.0390688017897
char_22: 0.0411577565609
char_23: 0.0369545764241
char_24: 0.0245830476445
char_25: 0.0419558187753
char_26: 0.0148824036234
char_27: 0.0276123033534
char_28: 0.0387835425398
char_29: 0.0176502836649
char_30: 0.022189949835
char_31: 0.0322979528096
char_32: 0.0391130346572
char_33: 0.0227434824364
char_34: 0.049567378195
char_35: 0.023148879453
char_36: 0.0504762355423
char_37: 0.0421365894057
year_x: 0.00159423144969
mo

In [9]:
def get_log_posterior2(train, name1, name2):
    n=len(train)
    aa=np.array(train[name1])
    bb=np.array(train[name2])
    ab=pd.Series([(aa[i], bb[i]) for i in np.arange(0, n)])
    s0=ab[np.array(train['outcome'] == 0)]
    s0=s0.append(pd.DataFrame({0:ab.unique()}))[0]
    s0=-np.log(s0.value_counts(dropna=False)/len(s0))
    s0=dict(s0)
    s1=ab[np.array(train['outcome'] == 1)]
    s1=s1.append(pd.DataFrame({0:ab.unique()}))[0]
    s1=-np.log(s1.value_counts(dropna=False)/len(s1))
    s1=dict(s1)
    return s0,s1

In [ ]:
field_pairs=set()
for name, name2 in itertools.combinations(train.columns.values, 2):
    non_categorical_fields=set(['people_id', 'activity_id', 'char_38'])
    if name not in non_categorical_fields and name2 not in non_categorical_fields:
        ie, lb = iteraction_entropy(train[name], train[name2], train['outcome'])
        if ie > 0.01:
            print(name, name2, end=': ')
            print(ie)
            field_pairs.add((name, name2))
            a[(name, name2)], b[(name, name2)] = get_log_posterior2(train, name, name2)

In [ ]:
def get_log_posterior_by_key(s, name, x):
    try:
        return s[name][x]
    except KeyError:
        return -999

In [ ]:
aa=np.zeros(len(test))
bb=np.zeros(len(test))
for name in test.columns.values:
    if name not in set(['people_id', 'activity_id', 'char_38']):
        aa = aa + np.array([get_log_posterior_by_key(a, name, x) for x in test[name]])
        bb = bb + np.array([get_log_posterior_by_key(b, name, x) for x in test[name]])

In [ ]:
submission=pd.DataFrame({'activity_id':act_test['activity_id'], 'outcome':np.argmin(np.concatenate((aa, bb)).reshape((2, len(aa))).T, 1)})

In [ ]:
submission.to_csv('submit.csv', index=False)

In [ ]:
aa, bb

In [6]:
np.random.seed(0)
folds_people=pd.DataFrame({'fold':np.random.randint(0, 5, len(people['people_id'].unique())), 'people_id': people['people_id'].unique()})
#folds=np.zeros(len(train))
cv=pd.merge(folds_people, train, on='people_id')
print([sum(cv['fold']!=0),sum(cv['fold']!=1),sum(cv['fold']!=2),sum(cv['fold']!=3),sum(cv['fold']!=4)])



[1733318, 1745884, 1796268, 1710044, 1803650]


In [7]:
np.random.seed(0)
folds_group_1=pd.DataFrame({'fold':np.random.randint(0, 5, len(people['group_1'].unique())), 'group_1': people['group_1'].unique()})
cv2=pd.merge(folds_group_1, train, on='group_1')
print([sum(cv2['fold']!=0),sum(cv2['fold']!=1),sum(cv2['fold']!=2),sum(cv2['fold']!=3),sum(cv2['fold']!=4)])


[1935284, 1943126, 1867439, 1906720, 1136595]


In [8]:
# create (sum(outcome), count) on the grid of group_1 x date
def create_outcome_group(train, namelist):
    outcome_group=train.groupby(namelist+['people_id'])['outcome']
    outcome_group=pd.DataFrame(outcome_group.mean()).reset_index()
    outcome_group=outcome_group.groupby(namelist)['outcome']
    return outcome_group

def create_group_pair(outcome_group):
    group_pair_1=dict(outcome_group.sum())
    group_pair_2=dict(outcome_group.count())
    group_pair = {key:(group_pair_1[key], group_pair_2[key]) for key in group_pair_2.keys()}    
    return group_pair

In [10]:
# create leave-one-person-out counting/proportion features 

def create_counting_features(thistrain):
    c=dict()
    people_list=thistrain['people_id'].unique()
    people_group=thistrain.groupby('people_id')
    for name in thistrain.columns.values:
        if name not in set(['people_id', 'activity_id', 'fold', 'char_38', 'outcome', 'date_x', 'date_y']) and \
            not name.startswith('extrapolate'):
                print(name, end=',')
                c_total=thistrain[name].value_counts(dropna=False)
                count_total=sum(c_total)
                c_people=people_group[name].value_counts(dropna=False)
                c_people=c_people.reset_index(name='counts')
                people_count_total=c_people.groupby('people_id')['counts'].sum()
                c_people['proportion']=c_people.apply(lambda x: (c_total[x[name]] - x['counts']) / (count_total - people_count_total[x['people_id']]), axis=1)
                c[name]=c_people.groupby(['people_id', name])['proportion'].mean()
    print('... [Done]')
    return c

print('Creating counting features for train ... ')
c=create_counting_features(cv)
print('Creating counting features for all ... ')
c_all=create_counting_features(pd.concat((train, test)))

Creating counting features for train ... 
char_1_x,group_1,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,char_8_x,char_9_x,char_10_x,char_11,char_12,char_13,char_14,char_15,char_16,char_17,char_18,char_19,char_20,char_21,char_22,char_23,char_24,char_25,char_26,char_27,char_28,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,year_x,month_x,day_x,weekday_x,activity_category,char_1_y,char_2_y,char_3_y,char_4_y,char_5_y,char_6_y,char_7_y,char_8_y,char_9_y,char_10_y,year_y,month_y,day_y,weekday_y,Creating counting features for all ... 
activity_category,char_10_x,char_10_y,char_11,char_12,char_13,char_14,char_15,char_16,char_17,char_18,char_19,char_1_x,char_1_y,char_20,char_21,char_22,char_23,char_24,char_25,char_26,char_27,char_28,char_29,char_2_x,char_2_y,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_3_x,char_3_y,char_4_x,char_4_y,char_5_x,char_5_y,char_6_x,char_6_y,char_7_x,char_7_y,char_8_x,char_8_y,char_9_x,char_9_y,day_x,day_y,grou

In [11]:
import collections
def create_od_group_pair(group_pair):
    a=collections.OrderedDict(sorted(group_pair.items()))
#    print(list(a.items())[0:20])
    return list(a.keys()), list(a.items())

In [12]:
%%cython

def get_outcome_mean_leaveoneout(group_pair, key_pair, value):
    try:
        a=group_pair[key_pair]
        if a[1]>1:
            return (a[0]-value)/(a[1]-1)
        else:
            return -999
    except:
        return -999
    
def get_outcome_mean_include(group_pair, key_pair):
    try:
        a=group_pair[key_pair]
        return a[0]/a[1]
    except:
        return -999
    
def get_outcome_mean(group_pair, row, namelist):
    return get_outcome_mean_include(group_pair[row['fold']], tuple(row[namelist]))

def get_outcome_mean_train(group_pair, row, namelist):
    return get_outcome_mean_leaveoneout(group_pair['train'], tuple(row[namelist]), row['outcome'])

def get_outcome_mean_test(group_pair, row, namelist):
    return get_outcome_mean_include(group_pair['train'], tuple(row[namelist]))

def get_outcome_mean1(group_pair, row, name):
    return get_outcome_mean_include(group_pair[row['fold']], row[name])

def get_outcome_mean1_train(group_pair, row, name):
    return get_outcome_mean_leaveoneout(group_pair['train'], row[name], row['outcome'])

def get_outcome_mean1_test(group_pair, row, name):
    return get_outcome_mean_include(group_pair['train'], row[name])

def get_outcome_mean2(group_pair, row, name1, name2):
    return get_outcome_mean_include(group_pair[row['fold']], (row[name1], row[name2]))

def get_outcome_mean2_train(group_pair, row, name1, name2):
    return get_outcome_mean_leaveoneout(group_pair['train'], (row[name1], row[name2]), row['outcome'])

def get_outcome_mean2_test(group_pair, row, name1, name2):
    return get_outcome_mean_include(group_pair['train'], (row[name1], row[name2]))

import bisect
def get_outcome_approxmean_leaveoneout(od_group_pair_keys, od_group_pair_items, row, namelist, datename, value):
    try:
        ind = bisect.bisect_left(od_group_pair_keys, tuple(row[namelist + [datename]]))
        twoside=0.
        count=0.
        backward=0.
        keys=tuple(row[namelist])
        date=row[datename]
        if od_group_pair_keys[ind][:-1] == keys and od_group_pair_keys[ind][-1] == date:
            a = od_group_pair_items[ind][-1]
            if a[1]>1:
                return (a[0] - value) / (a[1] - 1)
            else:
                backward=ind+1
        else:
            backward=ind
        forward=ind-1
        if od_group_pair_keys[backward][:-1] == keys:
            a = od_group_pair_items[backward][-1]
            weight=1/((od_group_pair_keys[backward][-1]-date).days+1)
            twoside = twoside + (a[0]/a[1])*weight
            count = count + weight
            
        if od_group_pair_keys[forward][:-1] == keys:
            a = od_group_pair_items[forward][-1]
            weight=1/((date-od_group_pair_keys[forward][-1]).days+1)
            twoside = twoside + (a[0]/a[1])*weight
            count = count + weight
        if count > 0:
            return twoside/count
        else:
            return -999
    except:
        return -999
    
def get_outcome_approxmean_include(od_group_pair_keys, od_group_pair_items, row, namelist, datename):
    try:
        ind = bisect.bisect_left(od_group_pair_keys, tuple(row[namelist + [datename]]))
        twoside=0.
        count=0.
        keys=tuple(row[namelist])
        date=row[datename]
        if od_group_pair_keys[ind][:-1] == keys:
            a = od_group_pair_items[ind][-1]
            weight=1/((od_group_pair_keys[ind][-1]-date).days+1)
            twoside = twoside + (a[0]/a[1])*weight
            count = count + weight
        if od_group_pair_keys[ind][-1] != date and od_group_pair_keys[ind-1][:-1] == keys:
            a = od_group_pair_items[ind-1][-1]
            weight=1/((date-od_group_pair_keys[ind-1][-1]).days+1)
            twoside = twoside + (a[0]/a[1])*weight
            count = count + weight
        if count > 0:
            return twoside/count
        else:
            return -999
    except:
        return -999

def get_outcome_approxmean(od_group_pair_keys, od_group_pair_items, row, name, datename):
    return get_outcome_approxmean_include(od_group_pair_keys[row['fold']], od_group_pair_items[row['fold']], row, name, datename) 

def get_outcome_approxmean_train(od_group_pair_keys, od_group_pair_items, row, name, datename):
    return get_outcome_approxmean_leaveoneout(od_group_pair_keys['train'], od_group_pair_items['train'], row, name, datename, row['outcome'])

def get_outcome_approxmean_test(od_group_pair_keys, od_group_pair_items, row, name, datename):
    return get_outcome_approxmean_include(od_group_pair_keys['train'], od_group_pair_items['train'], row, name, datename)

In [13]:
import tqdm
import collections
import bisect



def prebuild_feature(namelist):
    print('Process outcome group ... ')
    outcome_group=dict()
    for i in [0, 1, 2, 3, 4]:
        outcome_group[i]=create_outcome_group(train[cv['fold']!=i], namelist)
    outcome_group['train']=create_outcome_group(train, namelist)   
    print('Process group pair ...')
    group_pair=dict()
    for i in [0, 1, 2, 3, 4]:
        group_pair[i] = create_group_pair(outcome_group[i])
    group_pair['train']=create_group_pair(outcome_group['train'])
    print('Get Pairs ...')
    tqdm.tqdm_pandas(tqdm.tqdm_notebook())
    if len(namelist) > 2:
        thiscv=cv.progress_apply(lambda x: get_outcome_mean(group_pair, x, namelist), axis=1)
#        thistrain=train.progress_apply(lambda x: get_outcome_mean_train(group_pair, x, namelist), axis=1)
        thistest=test.progress_apply(lambda x: get_outcome_mean_test(group_pair, x, namelist), axis=1)
    elif len(namelist) == 2:
        thiscv=cv.progress_apply(lambda x: get_outcome_mean2(group_pair, x, namelist[0], namelist[1]), axis=1)
#        thistrain=train.progress_apply(lambda x: get_outcome_mean2_train(group_pair, x, namelist[0], namelist[1]), axis=1)
        thistest=test.progress_apply(lambda x: get_outcome_mean2_test(group_pair, x, namelist[0], namelist[1]), axis=1)         
    elif len(namelist) == 1:
        thiscv=cv.progress_apply(lambda x: get_outcome_mean1(group_pair, x, namelist[0]), axis=1)
#        thistrain=train.progress_apply(lambda x: get_outcome_mean1_train(group_pair, x, namelist[0]), axis=1)
        thistest=test.progress_apply(lambda x: get_outcome_mean1_test(group_pair, x, namelist[0]), axis=1)        
    return thiscv, thiscv, thistest
           
    
def prebuild_feature_with_date(namelist, datename, renamed_feature=None):
    print('Process outcome group ... ')
    outcome_group=dict()
    for i in [0, 1, 2, 3, 4]:
        outcome_group[i]=create_outcome_group(train[cv['fold']!=i], namelist + [datename])
    outcome_group['train']=create_outcome_group(train, namelist + [datename])
    print('Process group pair ...')
    group_pair=dict()
    for i in [0, 1, 2, 3, 4]:
        group_pair[i] = create_group_pair(outcome_group[i])
    group_pair['train']=create_group_pair(outcome_group['train'])
    print('Create ordered-dict group pair ...')
    od_group_pair_keys=dict()
    od_group_pair_items=dict()
    for i in [0, 1, 2, 3, 4]:
        od_group_pair_keys[i], od_group_pair_items[i]=create_od_group_pair(group_pair[i])
    od_group_pair_keys['train'], od_group_pair_items['train']=create_od_group_pair(group_pair['train'])
    print('Get Approximate Pairs ...')
    tqdm.tqdm_pandas(tqdm.tqdm_notebook())
    if renamed_feature is None:
        thiscv=cv.progress_apply(lambda x: get_outcome_approxmean(od_group_pair_keys, od_group_pair_items, x, namelist, datename), axis=1)
#        thistrain=train.progress_apply(lambda x: get_outcome_approxmean_train(od_group_pair_keys, od_group_pair_items, x, namelist, datename), axis=1)
        thistest=test.progress_apply(lambda x: get_outcome_approxmean_test(od_group_pair_keys, od_group_pair_items, x, namelist, datename), axis=1)
        return thiscv, thiscv, thistest              
    else:
        thiscv=cv.progress_apply(lambda x: \
                                 get_outcome_approxmean(od_group_pair_keys, od_group_pair_items, x, namelist, datename) \
                                 if x[renamed_feature]==-999 else x[renamed_feature], \
                                 axis=1)
#        thistrain=train.progress_apply(lambda x: \
#                                    get_outcome_approxmean_train(od_group_pair_keys, od_group_pair_items, x, namelist, datename) \
#                                    if x[renamed_feature]==-999 else x[renamed_feature], \
#                                    axis=1)
        thistest=test.progress_apply(lambda x: \
                                   get_outcome_approxmean_test(od_group_pair_keys, od_group_pair_items, x, namelist, datename) \
                                   if x[renamed_feature]==-999 else x[renamed_feature], \
                                   axis=1)
        return thiscv,thiscv,thistest

In [14]:
#cv['extrapolate'],train['extrapolate'], test['extrapolate'] = prebuild_feature(['group_1', 'date_y'])
#cv['extrapolate'].to_csv('cv:extrapolate.csv')
#train['extrapolate'].to_csv('train:extrapolate.csv')
#test['extrapolate'].to_csv('test:extrapolate.csv')
cv['extrapolate']=pd.Series.from_csv('cv:extrapolate.csv')
train['extrapolate']=pd.Series.from_csv('train:extrapolate.csv')
test['extrapolate']=pd.Series.from_csv('test:extrapolate.csv')


In [15]:
#cv['extrapolate2'], train['extrapolate2'], test['extrapolate2'] = prebuild_feature_with_date(['group_1'], 'date_y', renamed_feature='extrapolate')
#cv['extrapolate2'].to_csv('cv:extrapolate2.csv')
#train['extrapolate2'].to_csv('train:extrapolate2.csv')
#test['extrapolate2'].to_csv('test:extrapolate2.csv')
cv['extrapolate2']=pd.Series.from_csv('cv:extrapolate2.csv')
train['extrapolate2']=pd.Series.from_csv('train:extrapolate2.csv')
test['extrapolate2']=pd.Series.from_csv('test:extrapolate2.csv')

In [19]:
train['extrapolate']=cv['extrapolate']
train['extrapolate2']=cv['extrapolate2']

In [20]:
## what is interesting!
def basic_statistics(data):
    return sum(data[data['outcome']!=data['extrapolate2']]['extrapolate2']!=-999), \
           sum(np.array(data['extrapolate2']==-999))/len(data), \
           sum(np.array(data['extrapolate']==-999))/len(data)
basic_statistics(train)

(27593, 0.16515472916422996, 0.42693889885317876)

In [18]:
sum(np.array(test['extrapolate2']==-999))/len(test), \
sum(np.array(test['extrapolate']==-999))/len(test)

(0.1385899371750216, 0.41448443612927549)

In [21]:
a=dict()
b=dict()
gain=dict()
newtrain=cv[cv['extrapolate2']!=-999]
for name in newtrain.columns.values:
    if name not in set(['people_id', 'activity_id', 'char_38', 'fold', 'date', 'extrapolate', 'extrapolate2']):
        print(name, end=': ')
        gain[name] = mutual_entropy(newtrain[name], newtrain['outcome'])
        print(gain[name])
        a[name], b[name] = get_log_posterior(newtrain, name)

char_1_x: 0.0227136510419
group_1: 0.631496805867
char_2_x: 0.41449417444
date_x: 0.103450987051
char_3_x: 0.0276144299394
char_4_x: 0.0199715515455
char_5_x: 0.0188986677505
char_6_x: 0.0661623261049
char_7_x: 0.106231183299
char_8_x: 0.111716684544
char_9_x: 0.0998854108276
char_10_x: 0.0446959766188
char_11: 0.0319390700809
char_12: 0.0308145170995
char_13: 0.0651726132133
char_14: 0.0374490239961
char_15: 0.0437610770119
char_16: 0.0488039723483
char_17: 0.0518955318547
char_18: 0.0263508864808
char_19: 0.0494459694742
char_20: 0.0467037851608
char_21: 0.0489115301347
char_22: 0.0507054532148
char_23: 0.0472229083121
char_24: 0.0301810598637
char_25: 0.0525013414306
char_26: 0.0181899583187
char_27: 0.0340701607585
char_28: 0.0483006435597
char_29: 0.021235698689
char_30: 0.026307188195
char_31: 0.0395171475062
char_32: 0.0485386936629
char_33: 0.0272914856664
char_34: 0.0625867876657
char_35: 0.0279334970705
char_36: 0.0634833632596
char_37: 0.0524818163032
year_x: 0.0037851403231

In [ ]:
field_pairs=set()
for name, name2 in itertools.combinations(newtrain.columns.values, 2):
    non_categorical_fields=set(['people_id', 'activity_id', 'char_38', 'date_x', 'date_y', 'outcome', 'extrapolate', 'extrapolate2'])
    if name not in non_categorical_fields and name2 not in non_categorical_fields:
        ie, lb = iteraction_entropy(newtrain[name], newtrain[name2], newtrain['outcome'])
        if ie > 0.01:
            print(name, name2, end=': ')
            print(ie)
            field_pairs.add((name, name2))

In [ ]:
#train['extrapolate3'], test['extrapolate3'] = prebuild_feature_with_date(['char_3_x'], 'date_x')
#train['extrapolate3'].to_csv('train:extrapolate3.csv')
#test['extrapolate3'].to_csv('test:extrapolate3.csv')
train['extrapolate3']=pd.Series.from_csv('train:extrapolate3.csv')
test['extrapolate3']=pd.Series.from_csv('test:extrapolate3.csv')

In [ ]:
#train['extrapolate4'], test['extrapolate4'] = prebuild_feature(['char_7_x'])
#train['extrapolate4'].to_csv('train:extrapolate4.csv')
#test['extrapolate4'].to_csv('test:extrapolate4.csv')
train['extrapolate4']=pd.Series.from_csv('train:extrapolate4.csv')
test['extrapolate4']=pd.Series.from_csv('test:extrapolate4.csv')

In [ ]:
train['extrapolate5'], test['extrapolate5'] = prebuild_feature(['char_10_y'])
train['extrapolate5'].to_csv('train:extrapolate5.csv')
test['extrapolate5'].to_csv('test:extrapolate5.csv')

In [24]:
def create_train_data(train, test=None, c=None):
    train_feats=np.empty([len(train), 0])
    train_label=np.array(train['outcome'])
    feature_names=[];
    if test is not None:
        test_feats=np.empty([len(test), 0])
        
    for name in train.columns.values:
        if name not in set(['people_id', 'activity_id', 'fold', 'char_38', 'outcome', 'date_x', 'date_y']) and \
                            not str(name).startswith('extrapolate') and \
                            (gain[name] > 0.05 or name in set(['year_y','month_y'])):                        
            print(name, end=', ')
            feature_names=feature_names + [name]
            new_feat=np.array([c[name][train[['people_id', name]].to_records(index=False)]]).T            
            train_feats=np.append(train_feats, new_feat, 1)
            if test is not None:
                new_feat=np.array([c[name][test[['people_id', name]].to_records(index=False)]]).T          
                test_feats=np.append(test_feats, new_feat, 1)
        if name in set(['char_38', 'extrapolate2']):
            print(name, end=', ')
            feature_names=feature_names + [name]
            new_feat=np.array([train[name]], dtype=np.float).T
            train_feats=np.append(train_feats, new_feat, 1)
            if test is not None:
                new_feat=np.array([test[name]], dtype=np.float).T
                test_feats=np.append(test_feats, new_feat, 1)
    print(' ... [Done]')
    if test is None:
        return feature_names,train_feats,train_label
    else:
        return feature_names,train_feats,train_label,test_feats
    
feature_names,cv_feats,cv_label = create_train_data(cv, None, c)
feature_names,train_feats,train_label,test_feats = create_train_data(train, test, c_all)

group_1
char_2_x
char_6_x
char_7_x
char_8_x
char_9_x
char_13
char_17
char_22
char_25
char_34
char_36
char_37
char_38
char_10_y
year_y
month_y
extrapolate2
group_1
char_2_x
char_6_x
char_7_x
char_8_x
char_9_x
char_13
char_17
char_22
char_25
char_34
char_36
char_37
char_38
char_10_y
year_y
month_y
extrapolate2


In [26]:
def create_train_data2(train, test=None, c=None):
    train_feats=np.empty([len(train), 0])
    train_label=np.array(train['outcome'])
    feature_names=[];
    if test is not None:
        test_feats=np.empty([len(test), 0])
        
    for name in train.columns.values:
        if name not in set(['people_id', 'activity_id', 'fold', 'char_38', 'outcome', 'date_x', 'date_y']) and \
                            not str(name).startswith('extrapolate') and \
                            (gain[name] > 0.05 or name in set(['year_y','month_y'])):                        
            print(name, end=', ')
            feature_names=feature_names + [name]
            new_feat=np.array([c[name][train[['people_id', name]].to_records(index=False)]]).T            
            train_feats=np.append(train_feats, new_feat, 1)
            if test is not None:
                new_feat=np.array([c[name][test[['people_id', name]].to_records(index=False)]]).T          
                test_feats=np.append(test_feats, new_feat, 1)
        if name in set(['char_38']):
            print(name, end=', ')
            feature_names=feature_names + [name]
            new_feat=np.array([train[name]], dtype=np.float).T
            train_feats=np.append(train_feats, new_feat, 1)
            if test is not None:
                new_feat=np.array([test[name]], dtype=np.float).T
                test_feats=np.append(test_feats, new_feat, 1)
    print('... [Done]')
    if test is None:
        return feature_names,train_feats,train_label
    else:
        return feature_names,train_feats,train_label,test_feats  

feature_names2,cv2_feats,cv2_label = create_train_data2(cv2, None, c)
feature_names2,train2_feats,train2_label,test2_feats = create_train_data2(train, test, c_all)

group_1, char_2_x, char_6_x, char_7_x, char_8_x, char_9_x, char_13, char_17, char_22, char_25, char_34, char_36, char_37, char_38, char_10_y, year_y, month_y, ... [Done]
group_1, char_2_x, char_6_x, char_7_x, char_8_x, char_9_x, char_13, char_17, char_22, char_25, char_34, char_36, char_37, char_38, char_10_y, year_y, month_y, ... [Done]


In [27]:
eta = 0.05
gamma = 100
max_depth = 3
subsample = 0.8
colsample_bytree = 0.8

params = {
        "objective": "binary:logistic",
        "booster" : "gbtree",
        "eval_metric": "auc",
        "eta": eta,
        "gamma":gamma,
        "max_depth": max_depth,
#        "subsample": subsample,
#        "colsample_bytree": colsample_bytree,
        "silent": 1,
        'nthread': 4
}

In [28]:
import time
import random
import xgboost as xgb

from sklearn.cross_validation import LabelKFold


filter=np.array(cv['extrapolate']==-999) & np.array(cv['extrapolate2']!=-999)
fold=LabelKFold(np.array(cv['fold'])[filter], n_folds=5)
#fold=LabelKFold(np.array(cv['fold']), n_folds=5)


#weight=np.array(1./cv['people_id'].value_counts()[cv['people_id']])
dcv=xgb.DMatrix(cv_feats[filter], 
                cv_label[filter], missing=-999)

#alldtrain=xgb.DMatrix(train_feats, 
#                      train_label, missing=-999)

In [29]:
#watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
#gbm = xgb.train(params, dtrain, num_boost_round=200, evals=watchlist, early_stopping_rounds=3, verbose_eval=True)
model=xgb.cv(params, dcv, 200, folds=fold, 
             callbacks=[xgb.callback.print_evaluation(show_stdv=False),
                        xgb.callback.early_stop(10)])

[0]	train-auc:0.974301	test-auc:0.972834
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 10 rounds.
[1]	train-auc:0.975478	test-auc:0.973722
[2]	train-auc:0.976672	test-auc:0.974957
[3]	train-auc:0.977741	test-auc:0.976046
[4]	train-auc:0.977952	test-auc:0.975946
[5]	train-auc:0.978092	test-auc:0.976088
[6]	train-auc:0.978273	test-auc:0.976081
[7]	train-auc:0.978345	test-auc:0.976225
[8]	train-auc:0.978438	test-auc:0.976069
[9]	train-auc:0.978443	test-auc:0.976249
[10]	train-auc:0.978587	test-auc:0.976357
[11]	train-auc:0.978608	test-auc:0.976406
[12]	train-auc:0.97893	test-auc:0.97666
[13]	train-auc:0.979162	test-auc:0.97666
[14]	train-auc:0.979189	test-auc:0.976642
[15]	train-auc:0.979185	test-auc:0.97665
[16]	train-auc:0.980184	test-auc:0.978106
[17]	train-auc:0.980235	test-auc:0.978124
[18]	train-auc:0.980324	test-auc:0.978151
[19]	train-auc:0.980376	test-auc:0.978173
[20]	train-auc:0.980389	test-auc:

In [31]:
eta = 0.05
gamma = 10
max_depth = 6
subsample = 0.8
colsample_bytree = 0.8

params2 = {
        "objective": "binary:logistic",
        "booster" : "gbtree",
        "eval_metric": "auc",
        "eta": eta,
        "gamma":gamma,
        "max_depth": max_depth,
#        "subsample": subsample,
#        "colsample_bytree": colsample_bytree,
        "silent": 1,
        'nthread': 4
}


fold2=LabelKFold(np.array(cv2['fold']), n_folds=5)


weight2=np.array(1./cv2['group_1'].value_counts()[cv2['group_1']])
dcv2=xgb.DMatrix(cv2_feats, 
                 cv2_label, weight=weight2, missing=-999)

model2=xgb.cv(params2, dcv2, 200, folds=fold2, 
              callbacks=[xgb.callback.print_evaluation(show_stdv=False),
                         xgb.callback.early_stop(10)])

[0]	train-auc:0.740176	test-auc:0.731262
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 10 rounds.
[1]	train-auc:0.744467	test-auc:0.736467
[2]	train-auc:0.745226	test-auc:0.73711
[3]	train-auc:0.747581	test-auc:0.739703
[4]	train-auc:0.748178	test-auc:0.740349
[5]	train-auc:0.748868	test-auc:0.741038
[6]	train-auc:0.74943	test-auc:0.741701
[7]	train-auc:0.749902	test-auc:0.741978
[8]	train-auc:0.750271	test-auc:0.742486
[9]	train-auc:0.750632	test-auc:0.742856
[10]	train-auc:0.750799	test-auc:0.742836
[11]	train-auc:0.751174	test-auc:0.743056
[12]	train-auc:0.751943	test-auc:0.743478
[13]	train-auc:0.752206	test-auc:0.743587
[14]	train-auc:0.752923	test-auc:0.74407
[15]	train-auc:0.75338	test-auc:0.744218
[16]	train-auc:0.753598	test-auc:0.744229
[17]	train-auc:0.754012	test-auc:0.744645
[18]	train-auc:0.754448	test-auc:0.74498
[19]	train-auc:0.754687	test-auc:0.745271
[20]	train-auc:0.75497	test-auc:0.

In [32]:
filter=np.array(train['extrapolate']==-999) & np.array(train['extrapolate2']!=-999)
dtrain=xgb.DMatrix(train_feats[filter],
                   train_label[filter], missing=-999)

filter_test=np.array(test['extrapolate']==-999) & np.array(test['extrapolate2']!=-999)
dtest=xgb.DMatrix(test_feats[filter_test], missing=-999)

gbm1 = xgb.train(params, dtrain, num_boost_round=111, evals=[(dtrain, 'train')], verbose_eval=True)
print(gbm1.get_fscore())

submission1=pd.DataFrame({'activity_id':test['activity_id'][filter_test],'outcome': gbm1.predict(dtest)})
submission0=pd.DataFrame({'activity_id':test['activity_id'][test['extrapolate']!=-999],'outcome': test['extrapolate'][test['extrapolate']!=-999]})


[0]	train-auc:0.974603
[1]	train-auc:0.97527
[2]	train-auc:0.977138
[3]	train-auc:0.978153
[4]	train-auc:0.978226
[5]	train-auc:0.978155
[6]	train-auc:0.9783
[7]	train-auc:0.978301
[8]	train-auc:0.978218
[9]	train-auc:0.979233
[10]	train-auc:0.979232
[11]	train-auc:0.9793
[12]	train-auc:0.979289
[13]	train-auc:0.979373
[14]	train-auc:0.979349
[15]	train-auc:0.979344
[16]	train-auc:0.979424
[17]	train-auc:0.979349
[18]	train-auc:0.979378
[19]	train-auc:0.97939
[20]	train-auc:0.981722
[21]	train-auc:0.981722
[22]	train-auc:0.981723
[23]	train-auc:0.981745
[24]	train-auc:0.981741
[25]	train-auc:0.981812
[26]	train-auc:0.981852
[27]	train-auc:0.981926
[28]	train-auc:0.981923
[29]	train-auc:0.981937
[30]	train-auc:0.981935
[31]	train-auc:0.982242
[32]	train-auc:0.982794
[33]	train-auc:0.982879
[34]	train-auc:0.982922
[35]	train-auc:0.982936
[36]	train-auc:0.983052
[37]	train-auc:0.983107
[38]	train-auc:0.983063
[39]	train-auc:0.983067
[40]	train-auc:0.983107
[41]	train-auc:0.98311
[42]	trai

In [ ]:
weight2=np.array(1./train['group_1'].value_counts()[train['group_1']])
dtrain2=xgb.DMatrix(train2_feats,
                    train2_label, weight=weight2, missing=-999)

filter_test2=np.array(test['extrapolate2']==-999)
dtest2=xgb.DMatrix(test2_feats[filter_test2], missing=-999)

gbm2 = xgb.train(params2, dtrain2, num_boost_round=77, evals=[(dtrain2, 'train')], verbose_eval=True)
print(gbm2.get_fscore())

submission2=pd.DataFrame({'activity_id':test['activity_id'][filter_test2],'outcome': gbm2.predict(dtest2)})

[0]	train-auc:0.745994
[1]	train-auc:0.750388
[2]	train-auc:0.752023
[3]	train-auc:0.751655
[4]	train-auc:0.75302
[5]	train-auc:0.753209
[6]	train-auc:0.753737
[7]	train-auc:0.754086
[8]	train-auc:0.754474
[9]	train-auc:0.754739
[10]	train-auc:0.754674
[11]	train-auc:0.755266
[12]	train-auc:0.755592
[13]	train-auc:0.755638
[14]	train-auc:0.756121
[15]	train-auc:0.757045
[16]	train-auc:0.757373
[17]	train-auc:0.757843
[18]	train-auc:0.757836
[19]	train-auc:0.758538
[20]	train-auc:0.759144
[21]	train-auc:0.759503
[22]	train-auc:0.759788
[23]	train-auc:0.760062


In [293]:
submission=pd.concat((submission0, submission1, submission2))
submission.to_csv('submit_xgb_v4.csv', index=False)